# Task-2 Data Wrangling

The team wants to understand whether the “Apprehension Site Landmark” column could provide clues about potential partnerships between local and state law enforcement and ICE.

Use this column to help identify which **jail** or **prison** individuals have been taken to, where available. Add:
* one column to the dataset indicating the name of the facility
* another showing the county where that facility is located

## 0. Set up

In [1]:
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import re

import process_data

In [2]:
arrests_filename = 'arrests-0923-0625.csv'
cwd = Path.cwd()
root = cwd.parent
data = root / "data"

In [3]:
arrests_df = process_data.read_arrests_data(data/arrests_filename)

## 1 - Apprehension Site Landmark - data exploration

Quick scan of data to understand different formats in the field:

In [4]:
arrests_df['apprehension_site_landmark'].value_counts().head(20)

apprehension_site_landmark
DALLAS COUNTY GENERAL AREA                             11103
MTG GENERAL AREA, NON-SPECIFIC                          9009
NDD - 26 FEDERAL PLAZA NY, NY                           5803
HARRIS COUNTY JAIL, HOUSTON, TX                         4713
LOS ANGELES COUNTY GENERAL AREA, NON-SPECIFIC           4387
HLG GENERAL AREA, NON-SPECIFIC                          3535
ATLANTA, GA                                             3442
SNA GENERAL AREA, NON-SPECIFIC                          3357
AUS GENERAL AREA, NON-SPECIFIC                          2869
FUGITIVE OPERATIONS MA                                  2827
CAP - MARICOPA COUNTY SHERIFFS OFFICE JAIL              2732
MIRAMAR ICE/ERO SUB-OFFICE                              2465
WAS GENERAL AREA, NON-SPECIFIC                          2359
MIAMI DADE COUNTY JAIL TURNER GUILFORD KNIGHT (TGK)     2334
EDN GENERAL AREA, NON-SPECIFIC                          2227
ICE ERO NEWARK                                          21

A lot of `GENERAL AREA, NON-SPECIFIC`, which are not relevant for this task as they do not tell us about any partnerships or any jails or prisons. Removing them to get a better look at the types of variables in this field:

In [5]:
arrests_df[~arrests_df['apprehension_site_landmark'].fillna('').str.contains('GENERAL AREA, NON-SPECIFIC')]['apprehension_site_landmark'].value_counts().iloc[50:100]

apprehension_site_landmark
BENTON COUNTY JAIL 287(G)                              498
LIMESTONE COUNTY DETENTION CENTER, GROESBECK, TEXAS    493
FUGITIVE OPERATIONS NY STATE                           488
FTM-LEE COUNTY JAIL                                    475
FUGITIVE OPERATIONS CA STATE                           458
TRAVIS COUNTY JAIL, AUSTIN, TEXAS - TX2270000          433
TAM-POLK COUNTY JAIL                                   432
STUART-MARTIN COUNTY JAIL, FLORIDA                     416
FEDERAL PRISON LOMPOC MEDIUM                           407
UNION COUNTY JAIL                                      404
FEDERAL PRISON LOMPOC FCI II                           403
GWINNETT COUNTY JAIL                                   402
MCAT AZ STATE                                          401
WCD GENERAL AREA                                       400
HUDSON COUNTY JAIL                                     398
FAYETTE COUNTY CORRECTIONS, KY                         396
US PENITENTIARY THOMSON      

In [14]:
arrests_df[arrests_df['apprehension_site_landmark'].fillna('').str.contains('PRISON')]['apprehension_site_landmark'].value_counts().head(50)

apprehension_site_landmark
FEDERAL PRISON LOMPOC MEDIUM                                    407
FEDERAL PRISON LOMPOC FCI II                                    403
AVENAL STATE PRISON                                             300
PA STATE PRISON                                                 151
IRONWOOD STATE PRISON BLYTHE, CA                                111
US MEDICAL CENTER FOR FEDERAL PRISONERS, MISSOURI               106
PLEASANT VALLEY STATE PRISON                                     99
VALLEY STATE PRISON                                              86
CORCORAN STATE PRISON                                            84
HIGH DESERT STATE PRISON (NDOC)                                  78
CAP-DAUPHIN COUNTY PRISON PA                                     74
LIVINGSTON PARISH PRISON                                         69
ASCENSION PARISH PRISON                                          67
EAST BATON ROUGE PARISH PRISON                                   67
CHESTER COUNTY PRISON

In [18]:
arrests_df[arrests_df['apprehension_site_landmark'].fillna('').str.contains('JAIL')]['apprehension_site_landmark'].value_counts().iloc[50:100]

apprehension_site_landmark
HAMILTON COUNTY JAIL, TN                      206
MIDDLESEX COUNTY JAIL                         201
WEBER COUNTY JAIL - UT                        201
STUART-SAINT LUCIE COUNTY JAIL, FLORIDA       199
DAVIS COUNTY JAIL - UT                        196
ADAMS COUNTY JAIL                             195
HALL COUNTY JAIL - 287(G)                     194
HAYS COUNTY JAIL, SAN MARCOS, TEXAS           193
SUMNER COUNTY JAIL, TN                        184
CAP - JEFFERSON COUNTY JAIL, AL STATE         178
ESCAMBIA COUNTY JAIL                          168
FT BEND CO JAIL, RICHMOND, TX                 167
CAP - MADISON COUNTY JAIL, AL STATE           164
OAKLAND COUNTY JAIL, PONTIAC, MI              163
LUBBOCK COUNTY JAIL                           162
BOONE COUNTY JAIL, KY                         161
DENTON COUNTY JAIL                            160
GALVESTON CO JAIL, GALVESTON, TX              156
BERGEN COUNTY JAIL                            153
ORL - BREVARD COUNTY JA

#### Observations:

* Quite a lot of times where State is not given along with County - Counties in different States can have the same name, so will have to keep this in mind
* `ROCKINGHAM/HARRISONBURG REGIONAL JAIL` - `/` is a bit annoying, might need to clean that before extracting jail names
* Codes before jail name, e.g. `ORL - MARION COUNTY JAIL FLORIDA STATE` - do we want to capture these too? `CAP` means `Criminal Alien Program`, so other codes could be names of programs too
* Some state prisons only have abbreviations not names e.g. `PA STATE PRISON` instead of Pennsylvania

## 2 - Working out how to extract data 

#### 1. County:

In [6]:
county_expression = r"((?:\w+\s){0,3}\bCOUNTY)"

(expression worked out [here](https://regex101.com/r/OXnqFf/1))

In [20]:
prog = re.compile(county_expression)
result = prog.match('HALL COUNTY JAIL - 287(G)')

In [23]:
result.group(0)

'HALL COUNTY'

**But** - looks like some are abbreviated to county e.g. `FT BEND CO JAIL`, which is "Fort Bend County Jail"

Will see how common this is:

In [32]:
arrests_df[arrests_df['apprehension_site_landmark'].fillna('').str.contains(' CO ')]['apprehension_site_landmark'].value_counts()

apprehension_site_landmark
N DIST TX LUBBOCK DIV LUBBOCK CO NON CRIM         242
FT BEND CO JAIL, RICHMOND, TX                     167
GALVESTON CO JAIL, GALVESTON, TX                  156
BRAZORIA CO JAIL, ANGLETON, TX                    124
MINNEHAHA CO JAIL, SIOUX FALLS, SD                102
                                                 ... 
N DIST TX LUBBOCK DIV LYNN CO CRIM AT LARGE         1
N DIST TX AMARILLO DIV CARSON CO CRIM AT LARGE      1
HUGHES CO JAIL, PIERRE, SD                          1
N DIST TX AMARILLO DIV PARMER CO CRIM AT LARGE      1
N DIST TX LUBBOCK DIV LYNN CO P&P AT LARGE          1
Name: count, Length: 148, dtype: int64


Ok so definitely something to account for. Some of these don't seem to be jails/prisons though 

It looks like it's fine to replace 'CO' with 'COUNTY', and then treat them the same as the other 'COUNTY' entries:

In [10]:
arrests_df[
    (arrests_df['apprehension_site_landmark'].fillna('').str.contains(' CO ')) & 
    (arrests_df['apprehension_site_landmark'].fillna('').str.contains('JAIL'))
    ]['apprehension_site_landmark'].value_counts()

apprehension_site_landmark
FT BEND CO JAIL, RICHMOND, TX          167
GALVESTON CO JAIL, GALVESTON, TX       156
BRAZORIA CO JAIL, ANGLETON, TX         124
MINNEHAHA CO JAIL, SIOUX FALLS, SD     102
BRAZOS CO JAIL, BRYAN, TX              100
BROOKINGS CO JAIL, BROOKINGS, SD        33
DAVISON CO JAIL, MITCHELL, SD           28
JEFFERSON CO JAIL, BEAUMONT TX          27
CHAMBERS CO JAIL, ANAHUAC, TX           12
LAKE CO JAIL, MADISON, SD                9
FAYETTE CO JAIL, LA GRANGE, TX           7
BROWN CO JAIL, ABERDEEN, SD              5
NOBLES CO JAIL, WORTHINGTON, MN          5
BURLESON CO JAIL, CALDWELL, TX           5
COTTONWOOD CO JAIL, WINDOM, MN           4
ROBERTS CO JAIL, SISSETON, SD            3
DEWITT CO JAIL, CUERO, TX                3
MINER CO JAIL, HOWARD, SD                3
JEFFERSON CO JAIL, BEAUMONT, TX          2
CALHOUN CO JAIL, PORT LAVACA, TX         2
FAULK CO JAIL, FAULKTON, SD              2
TRIPP CO JAIL, WINNER, SD                1
HUGHES CO JAIL, PIERRE, SD 

In [11]:
arrests_df['apprehension_site_landmark'] = arrests_df['apprehension_site_landmark'].str.replace(' CO ', ' COUNTY ')

#### 2. Jail

Location information always seems to be before "JAIL", so can use the same expression

In [8]:
jail_expression = r"((?:\w+\s){0,3}\bJAIL)"

In [75]:
prog = re.compile(jail_expression)
result = prog.match('HALL COUNTY JAIL - 287(G)')

In [83]:
result.group(0)

'HALL COUNTY JAIL'

#### 3. Prison

Note - limitations and assumptions with this to explore in future work:
* Assumption that federal prisons are outside the scope of this project?
* Losing any location information that happens after "PRISON" - this could be improved in future work, but this appears to catch the majority of the cases

In [9]:
prison_expression = r"((?:\w+\s){0,3}\bPRISON)"

## 3- Extracting the relevant information

In [12]:
arrests_df['county'] = arrests_df['apprehension_site_landmark'].str.extract(county_expression)

**NOTE** - this is a bit of a clunky way to do it, if I get time I will come back to this, otherwise this is something that could be improved in the next stage of this work

In [13]:
arrests_df['jail'] = arrests_df['apprehension_site_landmark'].str.extract(jail_expression)

In [14]:
arrests_df['prison'] = arrests_df['apprehension_site_landmark'].str.extract(prison_expression)

In [15]:
arrests_df['facility'] = np.where(
                            ~arrests_df['jail'].isna(), arrests_df['jail'], np.where(
                            ~arrests_df['prison'].isna(), arrests_df['prison'], None))

In [16]:
arrests_df['county'] = np.where(
                        ~arrests_df['facility'].isna(), arrests_df['county'], None)

In [17]:
arrests_df.head()

,apprehension_date,apprehension_state,apprehension_aor,final_program,apprehension_method,apprehension_criminality,case_status,case_category,departed_date,departure_country,final_order_yes_no,birth_year,citizenship_country,gender,apprehension_site_landmark,unique_identifier,county,jail,prison,facility
0,2024-08-07 09:43:00,VIRGINIA,WASHINGTON AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,NON-CUSTODIAL ARREST,1 CONVICTED CRIMINAL,8-EXCLUDED/REMOVED - INADMISSIBILITY,[16] REINSTATED FINAL ORDER,2024-08-19,HONDURAS,YES,1981,HONDURAS,MALE,"HBG GENERAL AREA, NON-SPECIFIC",0000b34edd657d516c02b13a7c352d62d0effcb6,None,NaN,NaN,None
1,2024-10-19 08:33:00,TEXAS,HOUSTON AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP LOCAL INCARCERATION,1 CONVICTED CRIMINAL,6-DEPORTED/REMOVED - DEPORTABILITY,[16] REINSTATED FINAL ORDER,2024-10-22,MEXICO,YES,1984,MEXICO,MALE,"HARRIS COUNTY JAIL, HOUSTON, TX",0000ba6e459998a6046d185d82cf4349de1479d0,HARRIS COUNTY,HARRIS COUNTY JAIL,NaN,HARRIS COUNTY JAIL
2,2025-04-15 10:08:00,NEW JERSEY,NEWARK AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP FEDERAL INCARCERATION,1 CONVICTED CRIMINAL,8-EXCLUDED/REMOVED - INADMISSIBILITY,[16] REINSTATED FINAL ORDER,2025-06-10,DOMINICAN REPUBLIC,YES,1988,DOMINICAN REPUBLIC,MALE,"FORT DIX EAST, NEW JERSEY",0000c3d23fb0e444864559575900d410c4e8490f,None,NaN,NaN,None
3,2025-06-03 09:20:00,MINNESOTA,ST. PAUL AREA OF RESPONSIBILITY,FUGITIVE OPERATIONS,NON-CUSTODIAL ARREST,3 OTHER IMMIGRATION VIOLATOR,ACTIVE,[8G] EXPEDITED REMOVAL - CREDIBLE FEAR REFERRAL,NaT,NaN,YES,1985,COLOMBIA,FEMALE,"SPM GENERAL AREA, NON-SPECIFIC",0000d3dbf8033b5f209f6547ffee5b84feb4f599,None,NaN,NaN,None
4,2025-01-21 05:41:00,NaN,MIAMI AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP LOCAL INCARCERATION,2 PENDING CRIMINAL CHARGES,3-VOLUNTARY DEPARTURE CONFIRMED,[8C] EXCLUDABLE / INADMISSIBLE - ADMINISTRATIV...,2025-02-01,MEXICO,YES,1983,MEXICO,MALE,MIAMI DADE COUNTY JAIL TURNER GUILFORD KNIGHT ...,000104d730bf021326c6dc0deb3dd575304136b5,MIAMI DADE COUNTY,MIAMI DADE COUNTY JAIL,NaN,MIAMI DADE COUNTY JAIL


#### Extracting city, state with county, if it exists:

Would also be good to capture city, state info with the county if it exists, because e.g. there is a `KENT COUNTY JAIL` in Grand Rapids, MI, and in Maryland

First, checking whether we need to capture this information...

In [27]:
arrests_df.groupby('county')['apprehension_state'].nunique().sort_values()

county
MONTOUR COUNTY        0
AVERY COUNTY          0
WASHITA COUNTY        0
DEAF SMITH COUNTY     0
MARIN COUNTY          0
                     ..
LEE COUNTY            9
MADISON COUNTY        9
JEFFERSON COUNTY     10
FRANKLIN COUNTY      11
WASHINGTON COUNTY    14
Name: apprehension_state, Length: 899, dtype: int64

In [29]:
arrests_df[arrests_df['apprehension_site_landmark'].fillna('').str.contains('WASHINGTON COUNTY')]['apprehension_site_landmark'].unique()

array(['WASHINGTON COUNTY JAIL',
       'WASHINGTON COUNTY CORRECTIONAL FACILITY',
       'WASHINGTON COUNTY JAIL, UT',
       'WASHINGTON COUNTY SHERIFF, ILLINOIS',
       'WASHINGTON COUNTY JAIL, TN', 'WASHINGTON COUNTY JAIL, MN',
       'A - WASHINGTON COUNTY JAIL', 'WASHINGTON COUNTY JAIL ID',
       'A - WASHINGTON COUNTY GENERAL AREA',
       'WASHINGTON COUNTY PROBATION AND PAROLE',
       'WASHINGTON COUNTY FUGITIVE OPERATIONS',
       'WASHINGTON COUNTY DETENTION CENTER', 'WASHINGTON COUNTY MD',
       'WASHINGTON COUNTY JAIL, IA', 'WASHINGTON COUNTY, MS',
       'WASHINGTON COUNTY', 'A-WASHINGTON COUNTY JAIL',
       'WASHINGTON COUNTY GENERAL AREA ID', 'WASHINGTON COUNTY JAIL, NE'],
      dtype=object)

So here we can see there are multiple county jails called "Washington County Jail", one in Utah, Tennessee, Idaho, Minnesota etc. - so we do need to make sure we extract the information if it is there.

Note - data limitation - sometimes it just says "WASHINGTON COUNTY JAIL", without the relevant state information, so that could refer many different places

In [54]:
city_state_expression = r",(\s+\w.*$)"

(regex worked out [here](https://regex101.com/r/RTktg8/1))

In [55]:
re.search(city_state_expression, 'KENT COUNTY JAIL, GRAND RAPIDS, MI').group(0)

', GRAND RAPIDS, MI'

In [59]:
arrests_df['other_facility_loc_info'] = arrests_df['apprehension_site_landmark'].str.extract(city_state_expression)

In [62]:
arrests_df['other_facility_loc_info'] = np.where(
                        ~arrests_df['facility'].isna(), arrests_df['other_facility_loc_info'], None)

In [64]:
arrests_df.head()

,apprehension_date,apprehension_state,apprehension_aor,final_program,apprehension_method,apprehension_criminality,case_status,case_category,departed_date,departure_country,...,birth_year,citizenship_country,gender,apprehension_site_landmark,unique_identifier,county,jail,prison,facility,other_facility_loc_info
0,2024-08-07 09:43:00,VIRGINIA,WASHINGTON AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,NON-CUSTODIAL ARREST,1 CONVICTED CRIMINAL,8-EXCLUDED/REMOVED - INADMISSIBILITY,[16] REINSTATED FINAL ORDER,2024-08-19,HONDURAS,...,1981,HONDURAS,MALE,"HBG GENERAL AREA, NON-SPECIFIC",0000b34edd657d516c02b13a7c352d62d0effcb6,None,NaN,NaN,None,None
1,2024-10-19 08:33:00,TEXAS,HOUSTON AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP LOCAL INCARCERATION,1 CONVICTED CRIMINAL,6-DEPORTED/REMOVED - DEPORTABILITY,[16] REINSTATED FINAL ORDER,2024-10-22,MEXICO,...,1984,MEXICO,MALE,"HARRIS COUNTY JAIL, HOUSTON, TX",0000ba6e459998a6046d185d82cf4349de1479d0,HARRIS COUNTY,HARRIS COUNTY JAIL,NaN,HARRIS COUNTY JAIL,"HOUSTON, TX"
2,2025-04-15 10:08:00,NEW JERSEY,NEWARK AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP FEDERAL INCARCERATION,1 CONVICTED CRIMINAL,8-EXCLUDED/REMOVED - INADMISSIBILITY,[16] REINSTATED FINAL ORDER,2025-06-10,DOMINICAN REPUBLIC,...,1988,DOMINICAN REPUBLIC,MALE,"FORT DIX EAST, NEW JERSEY",0000c3d23fb0e444864559575900d410c4e8490f,None,NaN,NaN,None,None
3,2025-06-03 09:20:00,MINNESOTA,ST. PAUL AREA OF RESPONSIBILITY,FUGITIVE OPERATIONS,NON-CUSTODIAL ARREST,3 OTHER IMMIGRATION VIOLATOR,ACTIVE,[8G] EXPEDITED REMOVAL - CREDIBLE FEAR REFERRAL,NaT,NaN,...,1985,COLOMBIA,FEMALE,"SPM GENERAL AREA, NON-SPECIFIC",0000d3dbf8033b5f209f6547ffee5b84feb4f599,None,NaN,NaN,None,None
4,2025-01-21 05:41:00,NaN,MIAMI AREA OF RESPONSIBILITY,ERO CRIMINAL ALIEN PROGRAM,CAP LOCAL INCARCERATION,2 PENDING CRIMINAL CHARGES,3-VOLUNTARY DEPARTURE CONFIRMED,[8C] EXCLUDABLE / INADMISSIBLE - ADMINISTRATIV...,2025-02-01,MEXICO,...,1983,MEXICO,MALE,MIAMI DADE COUNTY JAIL TURNER GUILFORD KNIGHT ...,000104d730bf021326c6dc0deb3dd575304136b5,MIAMI DADE COUNTY,MIAMI DADE COUNTY JAIL,NaN,MIAMI DADE COUNTY JAIL,None


In [68]:
arrests_df[arrests_df['apprehension_site_landmark'].fillna('').str.contains('WASHINGTON COUNTY')][['apprehension_site_landmark','county','other_facility_loc_info']].drop_duplicates()

,apprehension_site_landmark,county,other_facility_loc_info
262,WASHINGTON COUNTY JAIL,WASHINGTON COUNTY,None
836,WASHINGTON COUNTY CORRECTIONAL FACILITY,None,None
2610,"WASHINGTON COUNTY JAIL, UT",WASHINGTON COUNTY,UT
7361,"WASHINGTON COUNTY SHERIFF, ILLINOIS",None,None
8011,"WASHINGTON COUNTY JAIL, TN",WASHINGTON COUNTY,TN
10827,"WASHINGTON COUNTY JAIL, MN",WASHINGTON COUNTY,MN
12875,A - WASHINGTON COUNTY JAIL,WASHINGTON COUNTY,None
14167,WASHINGTON COUNTY JAIL ID,WASHINGTON COUNTY,None
18520,A - WASHINGTON COUNTY GENERAL AREA,None,None
27096,WASHINGTON COUNTY PROBATION AND PAROLE,None,None


probably makes sense to combine the location info with the county name for ease of use in analysis:

In [70]:
arrests_df['county'] = arrests_df['county'] + arrests_df['other_facility_loc_info']

In [76]:
arrests_df.drop(['jail','prison', 'other_facility_loc_info'], axis=1)

arrests_df.to_csv(data/'arrests_with_facility_county.csv', index=False)

#### Next steps for this work

There is other potentially important information in Apprehension Site Landmark that could be extracted to add to the investigation:
* Potentially informative codes, e.g. CAP
* Other facilities not captured by Prison and Jail filters